## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E9E9BF6ED0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E9E9BF7980>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Mahendra and I am a Data Scientist")])

AIMessage(content="Hello Mahendra, it's nice to meet you!\n\nWhat can I do for you today? Are you working on any interesting projects? \n\nAs a large language model, I can help with a variety of tasks, such as:\n\n* **Generating code in different programming languages**\n* **Explaining complex concepts in data science**\n* **Summarizing research papers**\n* **Brainstorming ideas for new projects**\n* **Providing insights from text data**\n\n\nLet me know if there's anything I can assist you with!\n", response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 27, 'total_tokens': 143, 'completion_time': 0.241531245, 'prompt_time': 0.002609217, 'queue_time': None, 'total_time': 0.244140462}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6b69fdd-b33b-4e28-8db6-ee41b4da54a1-0', usage_metadata={'input_tokens': 27, 'output_tokens': 116, 'total_tokens': 143})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Mahendra and I am a Data Scientist"),
        AIMessage(content="Hello Mahendra! It's nice to meet you. \n\nAs a Data Scientist, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Mahendra, and you are a Data Scientist!  \n\nIs there anything else you'd like to tell me about your work?  Perhaps a project you're particularly proud of, or a challenge you're currently facing? \n\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 102, 'total_tokens': 156, 'completion_time': 0.110809199, 'prompt_time': 0.006661602, 'queue_time': None, 'total_time': 0.117470801}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2850a5cc-779b-41ee-a2ca-ad5bff9ef526-0', usage_metadata={'input_tokens': 102, 'output_tokens': 54, 'total_tokens': 156})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Mahendra and I am a Data Scientist")],
    config=config
)

In [9]:
response.content

"Hello Mahendra!\n\nIt's nice to meet you.  \n\nAs a fellow AI, I'm always interested in connecting with people in data science. What kind of work do you do as a Data Scientist? \n\nAre you working on any interesting projects right now?\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Mahendra. 😊  I remembered it from our first conversation! \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 101, 'total_tokens': 122, 'completion_time': 0.041846311, 'prompt_time': 0.006647342, 'queue_time': None, 'total_time': 0.048493653}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-be267013-fcc1-40c9-a54e-dc4f38648728-0', usage_metadata={'input_tokens': 101, 'output_tokens': 21, 'total_tokens': 122})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it! 😊  \n\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John Cena")],
    config=config1
)
response.content

"Well hello there, John Cena!  \n\nIt's a pleasure to meet you. What can I do for you today? 💪  😄 \n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"You said your name is John Cena!  \n\nDon't forget, I remember what you tell me. 😉\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Mahendra")]})

AIMessage(content="Hello Mahendra, it's nice to meet you!\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 37, 'total_tokens': 63, 'completion_time': 0.054381006, 'prompt_time': 0.002879867, 'queue_time': None, 'total_time': 0.057260873000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ca2ce96b-c1dc-47fb-af11-c990777a3862-0', usage_metadata={'input_tokens': 37, 'output_tokens': 26, 'total_tokens': 63})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Mahendra")],
    config=config
)

response

AIMessage(content="Hello Mahendra, it's nice to meet you!  \n\nIs there anything I can help you with today? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 37, 'total_tokens': 67, 'completion_time': 0.063438516, 'prompt_time': 0.002906297, 'queue_time': None, 'total_time': 0.066344813}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4876e6a-3e5c-4220-b787-3746be422d1e-0', usage_metadata={'input_tokens': 37, 'output_tokens': 30, 'total_tokens': 67})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Mahendra. 😊  \n\nI remember!  \n\nHow can I help you further?\n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Mahendra")],"language":"Hindi"})
response.content

'नमस्ते महेंद्र! 😊  \n\nआपको कैसे हो? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [37]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Mahendra")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते महेंद्र! 👋 \n\nआप कैसे हैं? 😊 \n'

In [38]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hinglish"},
    config=config,
)

In [39]:
response.content

"I don't know your name.  I'm a new AI, I don't have access to past conversations or any personal information about you. \n\nक्या आप अपना नाम बता सकते हैं? 😊 \n\n"

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\Mahendra Chourasiya\Projects\Models\Projects\day6\MY\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Mahendra Chourasiya\Projects\Models\Projects\day6\MY\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor?  🍦\n"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2" 😊  \n\n\n\nWould you like to try another one?\n'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n\n\n"

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"